0. ffill
1. medianfill
2. ebit
3. ttm
4. lags
5. dropna

In [1]:
# Patching sklearn for improved performance.
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# Importing necessary packages.
import pandas as pd
import numpy as np
from IPython.display import display
from preprocessing import *
from datetime import datetime
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv (r'/Users/Colin/hugo_research/final_data_v1.csv')
df['datadate'] = pd.to_datetime(df.datadate, format='%d/%m/%Y')
df.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,niq,ppegtq,rectq,revtq,txpq,uaptq,xoprq,xsgaq,costat,prccq
0,1000,1970-03-31,1970,1.0,INDL,C,D,STD,AE.2,000032102,...,0.605,NaN,NaN,9.478,NaN,NaN,NaN,NaN,I,NaN
1,1000,1970-06-30,1970,2.0,INDL,C,D,STD,AE.2,000032102,...,0.648,NaN,NaN,9.916,NaN,NaN,NaN,NaN,I,NaN
2,1000,1970-09-30,1970,3.0,INDL,C,D,STD,AE.2,000032102,...,0.369,NaN,NaN,10.698,NaN,NaN,NaN,NaN,I,11.75
3,1000,1970-12-31,1970,4.0,INDL,C,D,STD,AE.2,000032102,...,-1.064,NaN,NaN,10.919,NaN,NaN,35.749,8.249,I,10.00
4,1000,1971-03-31,1971,1.0,INDL,C,D,STD,AE.2,000032102,...,0.346,NaN,NaN,7.983,NaN,NaN,NaN,NaN,I,9.75


In [4]:
df.columns

Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'cusip', 'conm', 'curcdq', 'datacqtr', 'datafqtr',
       'acoq', 'aoq', 'cheq', 'cogsq', 'dlcq', 'dpq', 'invtq', 'lcoq', 'ltq',
       'niq', 'ppegtq', 'rectq', 'revtq', 'txpq', 'uaptq', 'xoprq', 'xsgaq',
       'costat', 'prccq'],
      dtype='object')

In [5]:
columns_list = df.columns.tolist().copy()
columns_list = columns_list[-19:]
columns_list.remove('costat')
columns_list.remove('prccq')                    #FIXME: still need to add 4 momenum features.
print(columns_list)

['acoq', 'aoq', 'cheq', 'cogsq', 'dlcq', 'dpq', 'invtq', 'lcoq', 'ltq', 'niq', 'ppegtq', 'rectq', 'revtq', 'txpq', 'uaptq', 'xoprq', 'xsgaq']


In [6]:
# Filling NaN forwardly.
df_ffill = ffill(df, columns_list)

# Filling NaN with cross-section median.
df_mfill = mfill(df_ffill, columns_list)

# Computing quartly EBIT as: ebitq = revtq - xoprq - dpq.
df_mfill['ebitq'] = df_mfill['revtq'] - df_mfill['xoprq'] - df_mfill['dpq']

# Converting quarterly data to trailing twelve months (TTM) data.
ttm_list = ['revtq', 'cogsq', 'xsgaq', 'niq', 'ebitq']
df_ttm = ttm(df_mfill, ttm_list)

# Adding lags to dataframe.                             #FIXME: shouldn't the lags be x4 here?
fundamentals_list = ['revtq', 'cogsq', 'xsgaq', 'niq', 'ebitq', 'cheq', 'rectq', 'invtq', 'acoq', 'ppegtq', 'aoq', 'dlcq', 'uaptq', 'txpq', 'lcoq', 'ltq']
df_lagged, X_col_list, y_col_list = lagged(df_ttm, fundamentals_list, lags_list=[-20, -16, -12, -8, -4, 4])

# Dropping rows with NaN.
df_ready = df_lagged.dropna()

In [7]:
print(df.shape)
print(df_ready.shape)

(624494, 33)
(226737, 130)


In [8]:
# Splitting dataframe into train (+val) and test sets.
df_train_val = df_ready[(df_ready.datadate >= datetime(1970, 1, 1)) & (df_ready.datadate <= datetime(1999, 12, 31))]
df_test = df_ready[(df_ready.datadate >= datetime(2000, 1, 1)) & (df_ready.datadate <= datetime(2016, 12, 31))]

In [9]:
# Splitting dataframes into X and y dataframes.
X_train_val = df_train_val.filter(X_col_list)
y_train_val = df_train_val.filter(y_col_list)
X_test = df_test.filter(X_col_list)
y_test = df_test.filter(y_col_list)

In [10]:
print(X_train_val.shape)
print(y_train_val.shape)

(106039, 96)
(106039, 16)


In [11]:
# Splitting train (+val) dataframe into training and validation sets.
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, train_size=0.7, random_state=42) #FIXME: not 30% of rows, but 30% of stocks!

In [12]:
# Scaling all features to have zero mean and unit standard deviation.           #FIXME: still need to do scaling on last time step!
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_val = scaler.transform(y_val)
y_test = scaler.transform(y_test)

In [13]:
# Saving all dataframes needed for modeling as pickle files.
df = [[X_train, X_val, X_test, y_train, y_val, y_test], ['X_train', 'X_val', 'X_test', 'y_train', 'y_val', 'y_test']]
for i in range(len(df[0])):
    name = str()
    pd.DataFrame(df[0][i]).to_pickle("preprocessing/" + str(df[1][i]) + ".pkl")